이제 최종적으로 데이터를 가져가서 사용할 일만 남았다. 이번에는 사용자나 고객이 인지하지 못하는 내부 시스템을 알아보자. 이는 흔히 백엔드 기술이라고도 한다. 스트리밍 API를 통해 클라이언트에 데이터를 보내는 작업이다.

# 핵심 개념

크게 3가지 카테고리로 나눠보자. 대시보드/애플리케이션, 서드파티 시스템/연동, 스트림 프로세서로 나눌 수 있으며 취합과 필터링은 모두 공통 요소로 들어간다. 

### UI/최종 사용자 애플리케이션

대시보드/애플리케이션은 처리된 스트림 데이터를 지속적으로 가져가는 역할을 한다. 이 애플리케이션은 간단한 웹브라우저 대시보드부터 메시징, 게임에 이르기까지 폭넓게 분포되어 있다. 일반적으로 애플리케이션은 스트림 서버와 직접 연동되고, 클라이언트 측에서 모든 스트림 데이터를 처리한다.

### 서드파티 시스템/연동, 스트림 프로세서

이 둘은 하나의 카테고리로 볼 수 있다. 이런 애플리케이션은 스트림 데이터를 가져가서 비즈니스 로직을 수행한 뒤 또 다른 시스템에 전달한다. 또 다른 시스템이란 외부 회사나 전혀 다른 애플리케이션이 될 수 있다. 예를 들어 고객 행동 데이터를 수집하는 시스템을 구축하고 구매 성향을 분석한 뒤 *SalesForce.com*에서 업데이트하는 것 등이 있다.  

대부분 회사 내부의 다른 시스템과 연동하는 스트림 프로세스 애플리케이션을 구축할 것이다. 소규모 서비스 간 통신을 수행해야 하는 분산 시스템 등에서 사용된다.

### 클라이언트 읽기 속도가 빠른 경우

경우에 따라 읽기 속도가 중요한 고려 사항이 될 수 있다. 스트리밍 API 츠면에서, 스트리밍 클라이언트 측면에서 클라이언트가 데이터를 빠르게 읽고 처리하는 것이 중요하다.  

스트리밍 API 측면에서 보면, 데이터 유실과 서버의 리소스 문제가 가장 크다. 처리되는 시간이 오래 걸리면 계속 전달되는 메시지를 어떻게 처리할지 적절하게 결정해야 하기 때문이다. 보통 메모리에 보관하는 방식이 좋아보이긴 하다. API 개발자는 서버 리소스가 고갈되지 않게 데이터를 일시적으로 폐기하거나 업스트림 시스템에 버퍼를 적용해야 할지도 모른다. 그리고 클라이언트에게 처리 지연이 발생하고 있다는 것을 알리는 기능이 필요할지도 모른다.  

그러면 클라이언트가 충분히 빨리 처리하고 있는지, 클라이언트가 종료되면 이 시스템이 어떻게 될지, 스트림 데이터 전송량이 많은 경우 클라이언트 성능을 높일면 어떻게 해야하는지, 어떻게 알 수 있을까? 이에 대한 해답은 서드파티 스트리밍 API를 이용하는지 혹은 내부 개발 API를 이용하는지에 따라 다르다.

##### 서드파티 스트리밍 API

트위터 API는 너무 처리가 늦는 클라이언트가 있으면 네트워크 연결을 끊어버린다. 또한 처리 속도가 늦는 경우 5분마다 지연 경고 메시지를 보낸다.  

서드파티 스트리밍 API가 이런 기능이 없다면 현재 메시지의 타임스탬프를 읽고 현재 시간과 비교하는 방법을 쓸 수 있다. 현재 시간과 메시지 시간이 차이가 나기 시작하면 클라이언트가 처리 속도가 늦는 이유를 찾을 수 있는 것이다. 물론 메시지에 타임 스탬프에 있어야 가능하다.  

타임 스탬프를 쓰지 않는다면 전달 속도를 예측하는 방법도 있다. 트위터는 클라이언트가 스트림 데이터를 충분히 빠르게 처리하지 못하면 연결을 종료할 것을 알리는데, 연결을 끊지 않고 뒤처진 데이터를 삭제하는 방식을 취할 수도 있다. 서드파티 스트리밍 API와 연동하는 클라이언트를 개발하고 있다면 이런 시나리오에서 어떻게 동작하는지 반드시 짚고 넘어가야 한다.

##### 자체 제작 스트리밍 API

스트리밍 소프트웨어 아키텍처 전체를 구축하는 경우, 전달되는 스트림 메시지 또는 로깅에 포함된 상태 메시지로 지연이 되고 있다는 것을 작성해 제공하는 것이 1가지 방법이다. 상태 메시지에는 클라이언트가 얼마나 처리 속도가 느린지 알려주고, 더 느리면 연결을 끊는다는 경고가 포함되어야 한다.  

### 데이터 처리 상태 관리

우선 사용자가 애플리케이션을 재시작했을 때 어떤 메시지부터 다시 처리하는지 알아야 한다. 그래서 마지막으로 처리한 메시지에 대한 메타데이터를 클라이언트에 저장하는 방법을 활용해 재시작 시 활용할 수 있다.

### 데이터 유실 줄이기

유실을 줄이기 위해 클라이언트도 주의할 부분이 있다.  

클라이언트 내부에서 유실되는 건 보통 구현을 잘못했을 때 벌어지는 이슈이기에, 이를 제외하고 데이터 송수신 과정에 발생하는 이슈를 살펴보자.  

데이터 유실을 막기 위해 HML을 적용할 수 있다. 클라이언트가 스트리밍 API 서버에서 데이터를 받은 다음 HML 방식으로 수신부를 구현한다. 클라이언트 내부에서 처리한 다음 서드파티로 데이터를 전송하기 전에 HML로 발신부를 구현하는 직관적인 방식이다. 

# 웹 클라이언트

서드파티와 달리 스림 데이터를 시각화하는데 많이 사용된다. 스트림 데이털르 상태 기반으로 관리해야 할 수도 있고, 클라이언트 자체적으로 데이터 처리를 해야 하는 경우도 있다. 스트림 데이터를 시각화하는 웹 클라이언트를 구축하면 회사의 비즈니스에 대한 이해도를 높이고 분석하기 용이하기에 스트리밍 클라이언트를 웹 애플리케이션으로 구현하는 것은 앞으로도 게속 발전할 것이다.  

스트림 데이터양이 많을 경우 웹 브라우저 기반 스트리밍 클라이언트의 처리 속도는 반드시 느려진다.  

예를 들어, 자바스크립트로 구현된 클라이언트는 데이터를 전달받아 브라우저의 로컬 스토리지에 저장한다. 